#### Evolution Node 
- Task, Input, Output
- LLM guided evolution
- LLM-engineered Code Function or Prompt

In [1]:
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.eoh_evolution import EvolNode

# Goal incoporation
# mp = MetaPrompt("Generate Fibonacci sequence.", "fibonacci", ["n"], ["sequence (list)"], PromptMode.CODE) # 
mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name_of_celebrity"], ["age"], PromptMode.PROMPT) # 

node = EvolNode(mp, None, None)

# Evolve the node (LLM guided function / prompt write-up)
reasoning, code = node._evolve("i1", replace=True)

# Now you can use this node as a proper function (!)
# node(10)
node("Deilireba")

Could not load vllm class, check CUDA support and GPU RAM size


{'name': 'Deilireba', 'age': 31}

#### Evolution Graph
- Take care of connecting each components (unfolding dictionary object etc.)
- Planning topology with evolution node

In [2]:
from methods.meta_prompt import MetaPlan, extract_json_from_text

mp = MetaPlan("Help me date Dilireba, I'm a white colar worker living on salary.")
prompt = mp._get_prompt_i1()
print("MetaPrompt I1: ")
print(prompt)

from methods.llm import get_openai_response
response = get_openai_response(prompt)

MetaPrompt I1: 
Goal: Help me date Dilireba, I'm a white colar worker living on salary.
First, describe the intuition for your tactics and main steps in one sentence. The description must be inside a brace.
Generate a JSON-style plan represented as a Directed Acyclic Graph (DAG) to achieve the goal. Use creative topology in the DAG, include parallel tasks if required.

The plan should include:

- **Nodes**: Each node represents a key action or step and must contain the following attributes:
  - `task`: Description of the task.
  - `name`: Concise name used for the task function.
  - `input`: The resources, information, or prerequisites needed to perform the action.
  - `output`: The immediate result or outcome of the action.
  - `target`: The purpose or goal that the action contributes to.
  - `mode`: The execution mode for this task ("CODE" or "PROMPT").

- **Edges**: Each edge represents a dependency or relationship between nodes, indicating that one step supports or leads to another

In [11]:
import re 
# from methods.meta_prompt import parse_json_from_response

tactic = re.findall(r"\{(.*)\}", response, re.DOTALL)
plan_dict = extract_json_from_text(response)

In [13]:
tactic

['Efficiently strategize to leverage personal strengths, shared interests, and social connections while maintaining genuine and respectful behavior.}\n\n```json\n{\n  "nodes": [\n    {\n      "task": "Research Dilireba\'s interests, hobbies, and public preferences.",\n      "name": "research_interests",\n      "input": "Internet, social media, celebrity interviews",\n      "output": "List of Dilireba\'s interests and hobbies",\n      "target": "Gathering knowledge on common interests",\n      "mode": "CODE"\n    },\n    {\n      "task": "Develop shared interest areas or hobbies.",\n      "name": "develop_shared_interests",\n      "input": "List of Dilireba\'s interests",\n      "output": "Personal engagement in shared hobbies",\n      "target": "Creating potential conversation and bonding points",\n      "mode": "CODE"\n    },\n    {\n      "task": "Improve personal appearance and fitness.",\n      "name": "self_improvement",\n      "input": "Gym membership, healthy diet plan, wardrobe

In [12]:
plan_dict

{'nodes': [{'task': "Research Dilireba's interests, hobbies, and public preferences.",
   'name': 'research_interests',
   'input': 'Internet, social media, celebrity interviews',
   'output': "List of Dilireba's interests and hobbies",
   'target': 'Gathering knowledge on common interests',
   'mode': 'CODE'},
  {'task': 'Develop shared interest areas or hobbies.',
   'name': 'develop_shared_interests',
   'input': "List of Dilireba's interests",
   'output': 'Personal engagement in shared hobbies',
   'target': 'Creating potential conversation and bonding points',
   'mode': 'CODE'},
  {'task': 'Improve personal appearance and fitness.',
   'name': 'self_improvement',
   'input': 'Gym membership, healthy diet plan, wardrobe upgrade',
   'output': 'Enhanced physical appearance and confidence',
   'target': 'Making a good first impression',
   'mode': 'CODE'},
  {'task': 'Network with people in the entertainment industry.',
   'name': 'network_industry',
   'input': 'Social media, indu

In [5]:
from methods.meta_prompt import MetaPrompt, MetaPlan, extract_json_from_text
from methods.eoh_evolution import EvolNode
from methods.llm import get_openai_response as get_response
from typing import Optional, List, Dict

# Collect MetaPrompt from parsed Plan-Graph Response
meta_node_prompts = []
for node in plan_dict["nodes"]:
    node_prompt = MetaPrompt(task=node.get("task"),  func_name=node.get("name"), input=node.get("input"), output=node.get("output"), mode=node.get("mode").lower())
    meta_node_prompts.append(node_prompt)
    
edges = plan_dict["edges"]

EvolNode(meta_prompt = node_prompt)


#### EvalNode Test: 
At the beginning level, generation of EvalNode (when code is used here) should compile with success, and it should takes input and produce output. 